In [1]:
import tensorflow as tf
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Layer, Conv2D, BatchNormalization, MaxPooling2D
from tensorflow.keras.layers import Dropout, Dense, Flatten, Input
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import OneHotEncoder
import random
import time

In [2]:
data_size = 3000
lr = 0.001
batch_size = 32
val_frac = 0.2
num_epochs = 50
random.seed(123)
np.random.seed(123)
tf.random.set_seed(123)

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train[:data_size].astype(np.float32)/255.0
y_train = y_train[:data_size]
x_test = x_test.astype(np.float32)/255.0
x_train = x_train.reshape((-1, 28, 28, 1))
x_test = x_test.reshape((-1, 28, 28, 1))

In [4]:
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)
y_train = OneHotEncoder(sparse=True).fit_transform(y_train).toarray()
y_test = OneHotEncoder(sparse=True).fit_transform(y_test).toarray()

In [5]:
def get_mean_std(images):
    mean = ()
    std = ()
    for i in range(images.shape[-1]):
        mean += (np.mean(images[:, :, :, i]),)
        std += (np.std(images[:, :, :, i]),)
    return mean, std

def normalize(images, mean, std):
    for i in range(images.shape[-1]):
        images[:, :, :, i] = (images[:, :, :, i] - mean[i])/std[i]
    return images

mean, std = get_mean_std(x_train)
x_train = normalize(x_train, mean, std)
x_test = normalize(x_test, mean, std)

In [6]:
perm_idx = np.random.permutation(x_train.shape[0])
val_idx = perm_idx[:int(val_frac*x_train.shape[0])]
train_idx = perm_idx[int(val_frac*x_train.shape[0]):]
x_val, y_val = x_train[val_idx], y_train[val_idx]
x_train, y_train = x_train[train_idx], y_train[train_idx]

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

In [17]:
def fit_gmm(losses):    
    losses = losses.numpy()
    losses = losses-losses.min()/(losses.max()-losses.min())
    input_loss = np.reshape(losses, [-1, 1])

    gmm = GaussianMixture(n_components=2, max_iter=50, tol=1e-2, reg_covar=5e-4)
    gmm.fit(input_loss)
    prob = gmm.predict_proba(input_loss)
    prob = prob[:, gmm.means_.argmin()]
    return prob

@tf.function
def test_step(x_batch, y_batch, loss_fn):    
    y_pred = model(x_batch)
    loss = loss_fn(y_pred, y_batch)
    y_pred_single = tf.math.argmax(model(x_batch, training=False), axis=1)
    y_batch_single = tf.math.argmax(y_batch, axis = 1)
    acc_metric.update_state(y_pred_single, y_batch_single)
    return loss

# def eval(model, val_ds, acc_metric, batch_size, losses):
def eval(model, val_ds, acc_metric, batch_size, len_val_ds):
    val_loss = 0
    losses = tf.Variable(tf.zeros(len_val_ds))
    loss_fn = tf.keras.losses.CategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    for idx, (x_batch, y_batch) in enumerate(val_ds):   
        loss = test_step(x_batch, y_batch, loss_fn)
        val_loss += tf.math.reduce_mean(loss)
        for b in tf.range(x_batch.shape[0]):
            # i = tf.cast(batch_size*idx, tf.int32)+b
            i = batch_size*idx+b
            # if i < losses.shape[0]:   
            losses[i].assign(tf.cast(loss[b], tf.float32)) 
    val_acc = acc_metric.result()
    acc_metric.reset_states()

    # fit loss to Gaussian mixture model
    # prob = tf.py_function(func=fit_gmm, inp=[losses], Tout=tf.float32)
    prob = fit_gmm(losses)
    
    return val_loss, val_acc, prob 

In [18]:
model = Sequential([
    Input(shape=(28, 28, 1)),
    Conv2D(10, kernel_size = 5, padding='same'),
    MaxPooling2D(pool_size = 2),
    Conv2D(20, kernel_size = 3, padding = 'same'),      
    Dropout(0.5),
    MaxPooling2D(pool_size = 2),
    Flatten(),
    Dense(30, activation = 'relu'),
    Dropout(0.5),
    Dense(10, activation = 'softmax')    
])

In [19]:
batch_size = 32
acc_metric = tf.keras.metrics.CategoricalAccuracy()
len_val_ds = len(y_val)

In [20]:
tf.keras.backend.clear_session()

model.compile(optimizer='adam', 
                loss = 'categorical_crossentropy',
                metrics = ['accuracy'])
for epoch in range(num_epochs):
    history = model.fit(train_ds, epochs=1)    
    # losses = tf.Variable(tf.zeros(len_val_ds))
    # losses = tf.zeros(len_val_ds)
    # val_loss, val_acc, _ = eval(model, val_ds, acc_metric, batch_size, losses)
    val_loss, val_acc, _ = eval(model, val_ds, acc_metric, batch_size, len_val_ds)
    print(f"Epoch {epoch+1} : val_loss : {val_loss}, val_acc : {val_acc}")

75/75 [==============================] - 1s 5ms/step - loss: 1.9677 - accuracy: 0.2850
Epoch 1 : val_loss : 221.46338953326145, val_acc : 0.6315789222717285
75/75 [==============================] - 0s 4ms/step - loss: 1.3984 - accuracy: 0.4896
Epoch 2 : val_loss : 168.5628434934964, val_acc : 0.6842105388641357
75/75 [==============================] - 0s 3ms/step - loss: 1.2011 - accuracy: 0.5562
Epoch 3 : val_loss : 128.8714358964935, val_acc : 0.6315789222717285
75/75 [==============================] - 0s 3ms/step - loss: 1.0216 - accuracy: 0.6288
Epoch 4 : val_loss : 99.69262322168409, val_acc : 0.7894737124443054
75/75 [==============================] - 0s 3ms/step - loss: 0.9037 - accuracy: 0.6796
Epoch 5 : val_loss : 90.9938499895264, val_acc : 0.7894737124443054
75/75 [==============================] - 0s 4ms/step - loss: 0.8160 - accuracy: 0.7017
Epoch 6 : val_loss : 71.19128402862286, val_acc : 0.7894737124443054
75/75 [==============================] - 0s 3ms/step - loss: 0.7

In [31]:
model.compile(optimizer='adam', 
                loss = 'categorical_crossentropy',
                metrics = ['accuracy'])
start_time = time.time()
history = model.fit(train_ds, validation_data = val_ds, epochs=50)
print(f"Training time : {(time.time()-start_time)/60:.2f} min.")


Epoch 1/50
75/75 [==============================] - 1s 7ms/step - loss: 1.9676 - accuracy: 0.2850 - val_loss: 1.4112 - val_accuracy: 0.6983
Epoch 2/50
75/75 [==============================] - 0s 4ms/step - loss: 1.4015 - accuracy: 0.4875 - val_loss: 0.9662 - val_accuracy: 0.8050
Epoch 3/50
75/75 [==============================] - 0s 4ms/step - loss: 1.1946 - accuracy: 0.5625 - val_loss: 0.6988 - val_accuracy: 0.8533
Epoch 4/50
75/75 [==============================] - 0s 4ms/step - loss: 1.0224 - accuracy: 0.6363 - val_loss: 0.5320 - val_accuracy: 0.8867
Epoch 5/50
75/75 [==============================] - 0s 3ms/step - loss: 0.9111 - accuracy: 0.6729 - val_loss: 0.4774 - val_accuracy: 0.8983
Epoch 6/50
75/75 [==============================] - 0s 3ms/step - loss: 0.8137 - accuracy: 0.7038 - val_loss: 0.4057 - val_accuracy: 0.9000
Epoch 7/50
75/75 [==============================] - 0s 4ms/step - loss: 0.7455 - accuracy: 0.7404 - val_loss: 0.3523 - val_accuracy: 0.9183
Epoch 8/50
75/75 [==